In [ ]:
%load_ext autoreload
%autoreload 2

import os

from polsarpro.dev.devtools import parse_psp_parameter_string
import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import read_psp_bin, open_netcdf_beam
from polsarpro.io import read_T3_psp
from polsarpro.decompositions import yamaguchi4
from polsarpro.dev.metrics import summarize_metrics
import shutil

# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths
# original dataset
input_alos_data = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")

# input files from C (use the same as for h-a-alpha)
input_test_dir = Path("/data/psp/SAN_FRANCISCO_ALOS1/")

# output files from C
output_test_dir = Path("/data/psp/res/yamaguchi4_c/")
if not os.path.isdir(output_test_dir):
    os.mkdir(output_test_dir)

## Run the C-version on some test data

In [ ]:
input_str = f"""id: {input_test_dir}
od: {output_test_dir}
iodf: S2
mod: Y4O
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
errf: /tmp"""

result = parse_psp_parameter_string(input_str)
os.system(f"yamaguchi_4components_decomposition.exe {result}")

# CPSP does not create another config file in output dir
shutil.copy(input_test_dir / "config.txt", output_test_dir)

## Load ALOS data and C outputs

In [ ]:
# uncomment to test on S matrix made with SNAP
S = open_netcdf_beam(input_alos_data)

## Apply the decomposition

In [ ]:
file_out = "/data/psp/res/test_yamaguchi4.nc"
# netcdf writer cannot overwrite
if os.path.isfile(file_out):
    os.remove(file_out)

# uncomment to test with PSP coherency
# T3 = read_T3_psp(input_test_dir)    

with ProgressBar():
    yamaguchi4(S, boxcar_size=[7, 7]).to_netcdf(file_out)


# Numerical evaluation

In [ ]:
import matplotlib.pyplot as plt

# open python output (comment if using compute)
out_py = xr.open_dataset("/data/psp/res/test_yamaguchi4.nc")
# open C-PolSARPro outputs
out_c = {}
out_names = (
    ("odd", "Yamaguchi4_Y4O_Odd"),
    ("double", "Yamaguchi4_Y4O_Dbl"),
    ("volume", "Yamaguchi4_Y4O_Vol"),
    ("helix", "Yamaguchi4_Y4O_Hlx"),
)
for name in out_names:
    file_name = output_test_dir / f"{name[1]}.bin"
    out_c[name[0]] = read_psp_bin(file_name)

df = summarize_metrics(out_py, out_c, short_titles=False, verbose=False)
df